In [1]:
import os
import random
import zipfile
import pandas as pd

import numpy as np
from skimage.io import imread  # reading images
from skimage.transform import resize  # resizing images
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

import tensorflow_datasets as tfds
from coders import LabelEncoder, DecodePredictions
from retinamodels import RetinaNet
from losses import RetinaNetLoss
from RetinaNetUtils import get_backbone, preprocess_data, resize_and_pad_image


In [2]:
url = "http://agristats.eu/images.zip"
filename = os.path.join(os.getcwd(), "images.zip")
keras.utils.get_file(filename, url)

with zipfile.ZipFile("images.zip", "r") as z_fp:
    z_fp.extractall("./")

In [3]:
model_dir = "retinanet/"
label_encoder = LabelEncoder()

num_classes = 80
batch_size = 2

learning_rates = [2.5e-06, 0.000625, 0.00125, 0.0025, 0.00025, 2.5e-05]
learning_rate_boundaries = [125, 250, 500, 240000, 360000]
learning_rate_fn = tf.optimizers.schedules.PiecewiseConstantDecay(
    boundaries=learning_rate_boundaries, values=learning_rates
)

In [4]:
resnet50_backbone = get_backbone()
loss_fn = RetinaNetLoss(num_classes)
model = RetinaNet(num_classes, resnet50_backbone)

optimizer = tf.optimizers.SGD(learning_rate=learning_rate_fn, momentum=0.9)
model.compile(loss=loss_fn, optimizer=optimizer)

In [5]:
callbacks_list = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(model_dir, "weights" + "_epoch_{epoch}"),
        monitor="loss",
        save_best_only=False,
        save_weights_only=True,
        verbose=1,
    )
]

import json
import io
# url = "http://agristats.eu/images.json"
# filename = os.path.join(os.getcwd(), "images.json")
# keras.utils.get_file(filename, url)
with open('images.json') as json_file:
    train_data = json.load(json_file)


In [6]:

directory = os.fsencode("images")
for file in os.listdir(directory):
  filename = os.fsdecode(file)
  for item in train_data:
    if filename == item["image/filename"]: 
      image = load_img(os.path.join("./images/", filename))
      image = img_to_array(image)
      item["image"] = image
      item["label"] = random.randint(0,1)
myimages = pd.DataFrame.from_dict(train_data).to_dict("list")
myimages = tf.data.Dataset.from_tensor_slices(myimages)

crop_size = 300
upscale_factor = 3
input_size = crop_size // upscale_factor


In [7]:
batch_size = 2
epochs = 5


In [8]:
image_count = (tf.data.experimental.cardinality(myimages).numpy())
image_count


220

In [9]:
train_dataset_count = .8 * image_count



In [10]:
train_dataset = myimages.take(train_dataset_count)
val_dataset = myimages.skip(train_dataset_count)


In [11]:
#before preprocessing
train_dataset_count = (tf.data.experimental.cardinality(train_dataset).numpy())
val_dataset_count = (tf.data.experimental.cardinality(val_dataset).numpy())
train_dataset_count, val_dataset_count


(176, 44)

In [12]:
# Uncomment the following lines, when training on full dataset
train_steps_per_epoch =  np.ceil((train_dataset_count/batch_size)-1)
# val_steps_per_epoch = \
#     dataset_info.splits["validation"].num_examples // batch_size


validation_steps_per_epoch = np.ceil((val_dataset_count/batch_size)-1)
# val_steps_per_epoch = \
#     dataset_info.splits["validation"].num_examples // batch_size
train_steps_per_epoch, validation_steps_per_epoch


(87.0, 21.0)

In [13]:
def getdatashape(dataset):
    for element in dataset:
     print(element[0].shape)
     break


In [14]:
autotune = tf.data.experimental.AUTOTUNE
train_dataset = train_dataset.map(preprocess_data, num_parallel_calls=autotune)
train_dataset = train_dataset.shuffle(8 * batch_size)

train_dataset = train_dataset.map(
    label_encoder.encode_batch, num_parallel_calls=autotune
)
train_dataset = train_dataset.apply(tf.data.experimental.ignore_errors())

train_dataset = train_dataset.repeat(epochs).batch(batch_size).prefetch(autotune)

getdatashape(train_dataset)
val_dataset = val_dataset.map(preprocess_data, num_parallel_calls=autotune)

val_dataset = val_dataset.map(
    label_encoder.encode_batch, num_parallel_calls=autotune)

val_dataset = val_dataset.apply(tf.data.experimental.ignore_errors())

val_dataset = val_dataset.repeat(epochs).batch(batch_size).prefetch(autotune)



samplebbox Tensor("args_1:0", shape=(4,), dtype=float32)
bbox Tensor("stack:0", shape=(1, 4), dtype=float32)
ασδ Tensor("concat:0", shape=(1, 4), dtype=float32) Tensor("mul_1:0", shape=(2,), dtype=float32)
samplebbox Tensor("args_1:0", shape=(4,), dtype=float32)
bbox Tensor("stack:0", shape=(1, 4), dtype=float32)
ασδ Tensor("concat:0", shape=(1, 4), dtype=float32) Tensor("mul_1:0", shape=(2,), dtype=float32)


In [15]:

# Running 100 training and 50 validation steps,
# remove `.take` when training on the full dataset

# Running 100 training and 50 validation steps,
# remove `.take` when training on the full dataset

model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=callbacks_list,
    verbose=1,
    steps_per_epoch=train_steps_per_epoch,
    validation_steps=validation_steps_per_epoch,
    batch_size=batch_size
)


Epoch 1/5


UnboundLocalError: local variable 'logs' referenced before assignment

In [ ]:
# Change this to `model_dir` when not using the downloaded weights
weights_dir = model_dir

latest_checkpoint = tf.train.latest_checkpoint(weights_dir)
model.load_weights(latest_checkpoint)

In [ ]:
image = tf.keras.Input(shape=[None, None, 3], name="image")
predictions = model(image, training=False)
detections = DecodePredictions(confidence_threshold=0.5)(image, predictions)
inference_model = tf.keras.Model(inputs=image, outputs=detections)

In [ ]:
def prepare_image(image):
    image, _, ratio = resize_and_pad_image(image, jitter=None)
    image = tf.keras.applications.resnet.preprocess_input(image)
    return tf.expand_dims(image, axis=0), ratio


val_dataset = tfds.load("coco/2017", split="validation", data_dir="data")
int2str = dataset_info.features["label"].int2str

image = load_img('test1.jpg')
image = img_to_array(image)

input_image, ratio = prepare_image(image)
detections = inference_model.predict(input_image)
num_detections = detections.valid_detections[0]
class_names = [
    int2str(int(x)) for x in detections.nmsed_classes[0][:num_detections]
]
visualize_detections(
    image,
    detections.nmsed_boxes[0][:num_detections] / ratio,
    class_names,
    detections.nmsed_scores[0][:num_detections],
)

#for sample in val_dataset.take(4):
#    print(sample["image"])
#    image = tf.cast(sample["image"], dtype=tf.float32)
#    input_image, ratio = prepare_image(image)
#    detections = inference_model.predict(input_image)
#    num_detections = detections.valid_detections[0]
#    class_names = [
#        int2str(int(x)) for x in detections.nmsed_classes[0][:num_detections]
#    ]
#    visualize_detections(
#        image,
#        detections.nmsed_boxes[0][:num_detections] / ratio,
#        class_names,
#        detections.nmsed_scores[0][:num_detections],
#    )
